# Running Auto Correlation with Talon Deployer and BITE
###### Last Updated: 18/03/24

This demo will show the basic operation of auto correlation using the new Deployer and BITE TANGO devices currently deployed in this repository. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

## Pre-requisites

First, this notebook assumes you have a running environment launched from a pipeline, in particular it assumes you are running off one launched from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline. Secondly, for ease of dev work, it also assumes you are using a virtual env. This notebook was made with Python 3.10 in mind.

Finally, make sure all requirements are installed via [poetry]().

### Running on Dev servers
To run this locally, a few extra steps are required:
* Uncomment the [pv.yaml file in Engineering Console](https://gitlab.com/ska-telescope/ska-mid-cbf-engineering-console/-/blob/main/charts/ska-mid-cbf-engineering-console-umbrella/templates/pv.yaml). This will establish the persistant volume, without having to go through MCS.
* Run the required make commands in the CICD repo.
    * `make minkube-clean DRIVER=docker`- Clean out the old minikube if previously used.
    * `make all DRIVER=docker` - Set up the minkube using the docker driver.
    * `eval $(minikube docker-env)` - Set the docker enivronment vairable.
* Then run the make commands to set up in the EC repo.
    * `make oci-build-all` - Build latest images/requriments in the repo.
    * `make oci-image-build` - Check to make sure the latest image is built.
    * `make k8s-reinstall-chart` - Deploy these images to the minikube.

If you check the minkube pods, you should see the TANGO database pods, the itango console and the old bite and deployer pods running. If they are still starting, give them a few moments.

In [12]:
!kubectl -n ska-mid-cbf get pods

NAME                                         READY   STATUS    RESTARTS      AGE
databaseds-ds-tango-databaseds-test-0        1/1     Running   2 (93s ago)   105s
databaseds-tangodb-tango-databaseds-test-0   1/1     Running   0             111s
ds-bite-bite-0                               1/1     Running   0             52s
ds-deployer-deployer-0                       1/1     Running   0             54s
ds-tangotest-test-0                          1/1     Running   0             53s
ska-tango-base-itango-console                1/1     Running   0             111s


This should return something like: 
```
NAME                                         READY   STATUS    RESTARTS   AGE
databaseds-ds-tango-databaseds-test-0        1/1     Running   0          17m
databaseds-tangodb-tango-databaseds-test-0   1/1     Running   0          17m
ds-bite-bite-0                               1/1     Running   0          16m
ds-deployer-deployer-0                       1/1     Running   0          16m
ds-tangotest-test-0                          1/1     Running   0          16m
ska-tango-base-itango-console                1/1     Running   0          17m
```

## Initial Setup/Using Deployer Device

First we want to set up the target board that we will be using for the auto correlation. While this can be multiple boards, for now we will only need one. Punch out the board(s) you have access to and set the `TARGET_BOARD` to assign it to be used in future steps.

The namespace should be left as is if you are running locally, otherwise it will need to pointed towards the ns generated by pipeline, e.g. `tango-databaseds-test.ci-ska-mid-psi-#####-USERNAME.svc.cluster.local:10000`.

In [13]:
import os
TARGET_BOARD = [3]
TANGO_HOST = "tango-databaseds-test.ska-mid-cbf.svc.cluster.local:10000"

os.environ["TANGO_HOST"] = TANGO_HOST

For this demo, we will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them as we would in the UI.

In [14]:
from PyTango import DeviceProxy, Database, exception

# Setup the device proxies targeting bite and deployer
db = Database()
deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")
bite_tango = DeviceProxy("mid_csp_cbf/ec/bite")
#Check the devices initially deployed to the database
devices = db.get_device_exported("*")
print(devices)
# Make sure the deployer device is set to ON
deployer_tango.On()
print("State of deployer device:")
print(deployer_tango.state())

DbDatum[
        name = '*'
value_string = ['dserver/DataBaseds/2', 'dserver/ECBite/bite', 'dserver/ECDeployer/deployer', 'dserver/TangoTest/test', 'mid_csp_cbf/ec/bite', 'mid_csp_cbf/ec/deployer', 'sys/database/2', 'sys/tg_test/1']]

State of deployer device:
ON


First, we set the target talon boards we want to set up our configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted.

In [15]:
deployer_tango.write_attribute("targetTalons",TARGET_BOARD)

With this set, we can then run the configuration command by calling the generate_config_jsons command.

In [16]:
deployer_tango.command_inout("generate_config_jsons")


We then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

**! CURRENTLY THIS DOES NOT WORK ON MINIKUBE, MUST USE PSI !**

In [8]:
deployer_tango.command_inout("download_artifacts", timeout=None)

CommunicationFailed: DevFailed[
DevError[
    desc = TRANSIENT CORBA system exception: TRANSIENT_CallTimedout
  origin = Connection::command_inout()
  reason = API_CorbaException
severity = ERR]

DevError[
    desc = Timeout (3000 mS) exceeded on device mid_csp_cbf/ec/deployer, command download_artifacts
  origin = Connection::command_inout()
  reason = API_DeviceTimedOut
severity = ERR]
]

In [8]:
!kubectl logs -n ska-mid-cbf ds-deployer-deployer-0

1|2024-03-18T21:53:45.099Z|INFO|MainThread|write_loggingLevel|base_device.py#1085|tango-device:mid_csp_cbf/ec/deployer|Logging level set to LoggingLevel.INFO on Python and Tango loggers
1|2024-03-18T21:53:45.100Z|INFO|MainThread|update_logging_handlers|base_device.py#313|tango-device:mid_csp_cbf/ec/deployer|Logging targets set to ['tango::logger']
1|2024-03-18T21:53:45.315Z|INFO|MainThread|exit|core.py#136||Finished processing state UNINITIALISED exit callbacks.
1|2024-03-18T21:53:45.315Z|INFO|MainThread|enter|core.py#130||Finished processing state INIT enter callbacks.
1|2024-03-18T21:53:45.315Z|INFO|MainThread|_update_state|base_device.py#905|tango-device:mid_csp_cbf/ec/deployer|Device state changed from UNKNOWN to INIT
1|2024-03-18T21:53:45.400Z|INFO|MainThread|callbacks|core.py#1147||Executed callback '<bound method OperationStateMachine._state_changed of <ska_tango_base.state_machine.OperationStateMachine object at 0x7f99486cfa00>>'
1|2024-03-18T21:53:45.400Z|INFO|MainThread|do|ba

Finally, we can configure the TANGO database with all the tango devices we just downloaded using the ConfigDB command.

In [9]:
deployer_tango.command_inout("configure_db")

DevFailed: DevFailed[
DevError[
    desc = FileNotFoundError: [Errno 2] No such file or directory: '/app/src/ska_mid_cbf_engineering_console/deployer/artifacts/fpga-talon/bin/talon_dx-tdc_base-tdc_vcc_processing.json'
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/server.py", line 1508, in cmd
    return get_worker().execute(f, self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/app/src/ska_mid_cbf_engineering_console/deployer/deployer_device.py", line 173, in configure_db
    configure_tango_db(json.loads(self._tango_db), self._logger)
  File "/app/src/ska_mid_cbf_engineering_console/deployer/midcbf_deployer.py", line 184, in configure_tango_db
    bitstream_f = open(
FileNotFoundError: [Errno 2] No such file or directory: '/app/src/ska_mid_cbf_engineering_console/deployer/artifacts/fpga-talon/bin/talon_dx-tdc_base-tdc_vcc_pr

The TANGO database should now be configured with all the devices needed for the next step, running the BITE device.

In [7]:
print(db.get_device_exported("*"))

DbDatum[
        name = '*'
value_string = ['dserver/DataBaseds/2', 'dserver/ECBite/bite', 'dserver/ECDeployer/deployer', 'dserver/TangoTest/test', 'mid_csp_cbf/ec/bite', 'mid_csp_cbf/ec/deployer', 'sys/database/2', 'sys/tg_test/1']]



## Running Commands Through the BITE Device